In [92]:
import pandas as pd
import exifread
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np

## Identify time deltas between images which are greater than expected.

In [131]:
imgdir = Path(r'D:\mylocalpath')
expected_freq_seconds = 1

In [132]:
def get_dt_original(fn):
    '''Gets DateTimeOriginal value from EXIF'''
    with open(fn, 'rb') as i:
        tags = exifread.process_file(i)
    dt_orig = str(tags.get('EXIF DateTimeOriginal'))
    return dt_orig

In [133]:
#instatiate new dataframe to hold image name, lat, long, etc
df = pd.DataFrame(columns=['ImageName','ImageDateTime'])

#get timestamps
for fn in imgdir.glob('**/*.*'):
    #Load image datetime from exif
    imgdt = datetime.strptime(get_dt_original(fn), '%Y:%m:%d %H:%M:%S')
    #Append to df
    df = df.append({
        'ImageName': fn.name,
        'ImageDateTime': imgdt
        }, ignore_index=True)

#get time delta
df['TimeDiff'] = df['ImageDateTime'].diff()

In [134]:
#Get min and max times
mintime = df[df['ImageDateTime'] == df['ImageDateTime'].min()]['ImageDateTime'].values[0]
maxtime = df[df['ImageDateTime'] == df['ImageDateTime'].max()]['ImageDateTime'].values[0]
timedelta = maxtime - mintime
seconds = timedelta.astype('timedelta64[s]').astype(np.int32)


#Report
print(f'First image time:              {np.datetime_as_string(mintime, unit="s")}')
print(f'Last image time:               {np.datetime_as_string(maxtime, unit="s")}')
print(f'Elapsed time in seconds:       {seconds}')
print(f'Number of images in directory: {df.shape[0]}\n')

print(f'Images preceeded by gaps greater than {expected_freq_seconds}')
df[df['TimeDiff'].dt.total_seconds() > expected_freq_seconds]


First image time:              2021-10-13T17:06:40
Last image time:               2021-10-13T17:06:45
Elapsed time in seconds:       5
Number of images in directory: 6

Images preceeded by gaps greater than 1


,ImageName,ImageDateTime,TimeDiff
